In [1]:
#include <iostream>

using namespace std;

# lambdas

Lambda are a syntax to construct a _function object_.

## review - function objects

- Consider a simple `struct` or `class` with an `invoke` member function

In [2]:
struct add_n_simple {
    int _n;
    
    int invoke(int x) const { return _n + x; }
};

- `add_n_simple` can be used as a function which adds `n` to any value passed to invoke

In [3]:
add_n_simple add_s{5};
cout << add_s.invoke(10) << "\n";

15


- We can make invocation more natural by providing a user-defined function call operator, `operator()`, instead of `invoke`
- This is known as a _function object type_

In [4]:
struct add_n_function {
    int _n;
        
    int operator()(int x) const { return _n + x; }
};

- Invocation now uses function calling syntax

In [5]:
add_n_function add_f{5};
cout << add_f(10) << "\n";

15


### anatomy of a function object

- _closure type_
```cpp
struct add_n_function {
```
- _captures_
```cpp
    int _n;
```
- _function call operator_
```cpp
    int operator()(int x) const { return _n + x; }
```
- _closure_
```cpp
add_n_function add_f{5};
```

## lambda expression

- A lambda expression is a prvalue of a unique unnamed class type, the _closure type_

In [6]:
{
    
auto add_n_lambda = [_n = 5](int x) -> int { return _n + x; };
cout << add_n_lambda(10) << "\n";
    
}

15


### anatomy of a lambda expression

- _closure type_
```cpp
    decltype(add_n_lambda)
```
- _captures_
```cpp
    [_n = 5]
```
- _function call operator_
```cpp
    (int x) -> int { return _n + x; }
```
- _closure_
```cpp
    add_n_lambda
```